# Load Training Data 

In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras import Sequential

img_height = 256
img_width = 192
batch_size = 16 #number of samples processed before the model is updated

n_train = '[Path to File]'

datagen = ImageDataGenerator(
    rescale=1.0 / 255, #standardise data
    horizontal_flip=False,
    vertical_flip=False,
    validation_split=0.2,
)

train_generator = datagen.flow_from_directory(
    directory = n_train,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    color_mode="rgb",
    class_mode="categorical",
    shuffle=True,
    #subset="training",
    seed=123 
)



# VGG16 Layers Preparation

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras import Sequential




model = VGG16(weights="imagenet")

base_model = VGG16(weights="imagenet",include_top=False)

print(base_model.summary())

x = base_model.output # base model (pretrained)

# Add few layers
x = GlobalAveragePooling2D()(x) 

x = Dense(1024, activation='relu')(x)
x = Dense(1024, activation='relu')(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
preds = Dense(2, activation='softmax')(x)

model = tf.keras.models.Model(inputs=base_model.input, outputs=preds)

print(model.summary())

In [ ]:
for i, layer in enumerate(model.layers):
  print(i, layer.name)

# Layers Freezing

In [ ]:
for layer in model.layers[:19]: #to avoid training existing weights // to avoid training trained layers
    layer.trainable = False

for layer in model.layers[19:]: #train 19th layer and above *because it is not yet trained*
    layer.trainable = True

# Training

In [ ]:
#tf.device('/gpu:1') #enable gpu so my machine does not blows up

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

# Redo model.compile to reset the optimizer states
model.compile(
    #optimizer=keras.optimizers.Adam(),
    #loss=[keras.losses.SparseCategoricalCrossentropy(from_logits=True),],
    optimizer=tf.optimizers.Adam(learning_rate=0.0001),
    loss="categorical_crossentropy",
    metrics=["accuracy", tf.keras.metrics.Precision(), tf.keras.metrics.Recall()],
)

# using model.fit (note steps_per_epoch)
history = model.fit(
    train_generator,
    epochs=15,
    verbose=1,
    #steps_per_epoch=n_train/batch_size,
    #validation_steps=n_valid/batch_size,
)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history["accuracy"])
plt.plot(history.history['loss'])
plt.title("model accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Accuracy","loss"])
fig = plt.gcf()
plt.show()


# Load Test Data

In [ ]:
img_height = 256
img_width = 192
batch_size = 16 #number of samples processed before the model is updated

In [ ]:
datagen = ImageDataGenerator(
    rescale=1.0 / 255, #standardise data
    horizontal_flip=False,
    vertical_flip=False,
    validation_split=0.2,
)

In [ ]:
test_generator = datagen.flow_from_directory(
    directory = '/Users/User/Desktop/FYP/Dataset/apple/test',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    color_mode="rgb",
    class_mode="categorical",
    shuffle=False,
    #subset="testing",
    seed=123 
)

In [ ]:
#evaluate the model
test_hist = model.evaluate(
    test_generator
)

# Prediction

In [ ]:
predict = model.predict(test_generator) #predict the whole test dataset

# Testing and Evaluation

### Confusion Matrix

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix,classification_report


y_pred = np.argmax(predict, axis=1)

true_classes = test_generator.classes

class_labels = list(test_generator.class_indices.keys())   

print("\tCONFUSION MATRIX")
print("\n")
print(confusion_matrix(test_generator.classes, y_pred, ))
print("\n")

### Classification Report

In [ ]:
print("\t\t\tCLASSIFICATION REPORT")
print("\n")
report = classification_report(true_classes, y_pred, target_names=class_labels.sort())
print(report)